## Few Business questions that we are trying to answer with this data:
#### 1. Does most favoritized tweet has any pattern/ trend? <br> 2. Which countries are most people tweeting from? <br> 3. Most favoritized tweets? <br> 4. Most followed people and their description? <br> 5. What is the sentiment in peoples' tweet? <br> 6. Does the sentiment has any correlation with country?

## Import Libraries and Data Load

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import os
import string
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

In [ ]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(os.path.join(dirname, filename))
df.head()

## Exploration Data Analysis (EDA)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.corr())

## Detailed Analysis

In [ ]:
# Make a copy of dataframe before making any changes
tweets = df.copy()

In [ ]:
# Convert date column to datetime
tweets['date'] = pd.to_datetime(tweets['date'])

In [ ]:
# Take care of nulls in location and description
tweets.user_location.fillna('Unknown', inplace=True)
tweets.user_description.fillna('Unknown', inplace=True)
tweets.source.fillna('Unknown', inplace=True)
tweets.hashtags.fillna('None', inplace=True)

In [ ]:
# Verify
tweets.info()

In [ ]:
tweets.user_location.value_counts().to_frame().style.bar()

In [ ]:
# Attempt tp clean the location column. There are many other business rules that can be applied to 
# improve the accuracy of different cases. We can also use regular expressions

# If both country and city is mentioned

tweets["country"] = tweets.user_location.apply(lambda x: x.split(",")[-1].strip() \
                                            if ("," in x) else x)
tweets["city"] = tweets.user_location.apply(lambda x: x.split(",")[0].strip() \
                                            if ("," in x) else x)

# Replacing 2 digit US states with USA except UK
tweets["country"] = tweets.country.apply(lambda x: 'USA' if len(x.lower().strip())<3 and x!='uk' else x)

# Standarizing case senstive cases
tweets["country"] = tweets.country.apply(lambda x: 'USA' if x.lower().strip() in \
                                      ("united states, usa, us") else x)
tweets["country"] = tweets.country.apply(lambda x: 'India' if x.lower().strip() in \
                                      ("india") else x)

In [ ]:
tweets.country.value_counts().to_frame().style.bar()

## We are able to confirm that US and India are the top countries from where most of the people have tweeted with USA being significantly higher

In [ ]:
tweets["city"] = tweets.city.apply(lambda x: 'Unknown' if x.lower() in \
                                   ['india', 'united states', 'united kingdom', 'uk', 'usa', 'us'] \
                                   else x)

In [ ]:
tweets.city.value_counts().to_frame().style.bar()

## The top 5 cities with highest tweets are seen to be London, New Delhi, New York, Mumbai, Washington.

In [ ]:
# We can see a lot of cases have been taken care of but of course there are many additional business 
# rules that can be applied to improve the accuracy but for now this should be sufficient
tweets[['user_location','country', 'city']].head(100)

In [ ]:
tweets.info()

In [ ]:
# five largest values in column tweets 
tweets.nlargest(5, ['user_favourites']) 

## Check sentiment of tweets

In [ ]:
# Normalize the text, convert everything other than a-z, A-Z, 0-9 
#text = re.sub(r"[^a-zA-Z0-9]"," ",tweets['text'])

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
words = word_tokenize(tweets['text'])